In [1]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


In [2]:
Lx, Lz = 1, 2
Nx, Nz = 128, 256
Reynolds = 5e4
Schmidt = 1
stop_sim_time = 1
timestepper = d3.RK222
max_timestep = 1e-2
dtype = np.float64
dealias = 5

In [3]:
coords = d3.CartesianCoordinates('x', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
zbasis = d3.RealFourier(coords['z'], size=Nz, bounds=(-Lz/2, Lz/2), dealias=dealias)

# Fields
p = dist.Field(name='p', bases=(xbasis,zbasis))
s = dist.Field(name='s', bases=(xbasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,zbasis))
tau_p = dist.Field(name='tau_p')

In [4]:
# Substitutions
nu = 1 / Reynolds
D = nu / Schmidt
x, z = dist.local_grids(xbasis, zbasis)
ex, ez = coords.unit_vector_fields(dist)

# Problem
problem = d3.IVP([u, s, p, tau_p], namespace=locals())
problem.add_equation("dt(u) + grad(p) - nu*lap(u) = - u@grad(u)")
problem.add_equation("dt(s) - D*lap(s) = - u@grad(s)")
problem.add_equation("div(u) + tau_p = 0")
problem.add_equation("integ(p) = 0") # Pressure gauge


{'LHS': Integrate(Integrate(<Field 140685679981712>)),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(<Field 140685679981712>)),
 'F': <Field 140685684010768>,
 'domain': <dedalus.core.domain.Domain at 0x7ff3efbda710>,
 'matrix_dependence': array([ True,  True]),
 'matrix_coupling': array([False, False])}

In [5]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time

# Initial conditions
# Background shear
u['g'][0] = 1/2 + 1/2 * (np.tanh((z-0.5)/0.1) - np.tanh((z+0.5)/0.1))
# Match tracer to shear
s['g'] = u['g'][0]
# Add small vertical velocity perturbations localized to the shear layers
u['g'][1] += 0.1 * np.sin(2*np.pi*x/Lx) * np.exp(-(z-0.5)**2/0.01)
u['g'][1] += 0.1 * np.sin(2*np.pi*x/Lx) * np.exp(-(z+0.5)**2/0.01)

# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.1, max_writes=10)
snapshots.add_task(s, name='tracer')
snapshots.add_task(p, name='pressure')
snapshots.add_task(-d3.div(d3.skew(u)), name='vorticity')
snapshots.add_task(u, name="velocity")
snapshots.add_task(u, name="velocity_c", layout="c")


# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.2, threshold=0.1,
             max_change=1.5, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property((u@ez)**2, name='w2')

2023-05-25 17:46:01,015 subsystems 0/1 INFO :: Building subproblem matrices 1/64 (~2%) Elapsed: 0s, Remaining: 4s, Rate: 1.8e+01/s
2023-05-25 17:46:01,168 subsystems 0/1 INFO :: Building subproblem matrices 7/64 (~11%) Elapsed: 0s, Remaining: 2s, Rate: 3.4e+01/s
2023-05-25 17:46:01,351 subsystems 0/1 INFO :: Building subproblem matrices 14/64 (~22%) Elapsed: 0s, Remaining: 1s, Rate: 3.6e+01/s
2023-05-25 17:46:01,581 subsystems 0/1 INFO :: Building subproblem matrices 21/64 (~33%) Elapsed: 1s, Remaining: 1s, Rate: 3.4e+01/s
2023-05-25 17:46:01,762 subsystems 0/1 INFO :: Building subproblem matrices 28/64 (~44%) Elapsed: 1s, Remaining: 1s, Rate: 3.5e+01/s
2023-05-25 17:46:01,991 subsystems 0/1 INFO :: Building subproblem matrices 35/64 (~55%) Elapsed: 1s, Remaining: 1s, Rate: 3.4e+01/s
2023-05-25 17:46:02,226 subsystems 0/1 INFO :: Building subproblem matrices 42/64 (~66%) Elapsed: 1s, Remaining: 1s, Rate: 3.3e+01/s
2023-05-25 17:46:02,418 subsystems 0/1 INFO :: Building subproblem matri

In [6]:
# Main loop
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_w = np.sqrt(flow.max('w2'))
            logger.info('Iteration=%i, Time=%e, dt=%e, max(w)=%f' %(solver.iteration, solver.sim_time, timestep, max_w))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()

2023-05-25 17:46:03,042 __main__ 0/1 INFO :: Starting main loop
2023-05-25 17:46:05,090 __main__ 0/1 INFO :: Iteration=1, Time=1.000000e-02, dt=1.000000e-02, max(w)=0.100000
2023-05-25 17:46:10,343 __main__ 0/1 INFO :: Iteration=11, Time=1.100000e-01, dt=1.000000e-02, max(w)=0.040690
2023-05-25 17:46:14,942 __main__ 0/1 INFO :: Iteration=21, Time=1.600000e-01, dt=5.000000e-03, max(w)=0.040906
2023-05-25 17:46:19,022 __main__ 0/1 INFO :: Iteration=31, Time=1.905206e-01, dt=3.052063e-03, max(w)=0.041092
2023-05-25 17:46:23,495 __main__ 0/1 INFO :: Iteration=41, Time=2.210413e-01, dt=3.052063e-03, max(w)=0.041307
2023-05-25 17:46:27,966 __main__ 0/1 INFO :: Iteration=51, Time=2.515619e-01, dt=3.052063e-03, max(w)=0.041561
2023-05-25 17:46:33,062 __main__ 0/1 INFO :: Iteration=61, Time=2.820825e-01, dt=3.052063e-03, max(w)=0.041857
2023-05-25 17:46:38,423 __main__ 0/1 INFO :: Iteration=71, Time=3.126032e-01, dt=3.052063e-03, max(w)=0.042196
2023-05-25 17:46:43,729 __main__ 0/1 INFO :: Iter